In [ ]:
!curl -b session=$(cat ${HOME}/advent_of_code_session) https://adventofcode.com/2019/day/2/input -o input

In [21]:
from dataclasses import dataclass
from typing import Callable
import numpy as np
from functools import partial

In [22]:
with open("input", "r") as file:
    lines = file.read().split()

In [24]:
class ProgramStop(Exception):
    def __init__(self, final_memory):
        super().__init__()
        self.state = final_memory

@dataclass
class OpCode:
    code: int
    n_args: int
    func: Callable

def exit(memory):
    raise ProgramStop(memory)

def arithmetic(memory, a, b, out, op):
    memory[out] = op(memory[a], memory[b])

interpreter = {
    op.code: op for op in [
        OpCode(code=1, n_args=3, func=partial(arithmetic, op=lambda a, b: a + b)),
        OpCode(code=2, n_args=3, func=partial(arithmetic, op=lambda a, b: a * b)),
        OpCode(code=99, n_args=0, func=exit)
    ]
}

In [25]:
def simulate(program):
    memory = program.copy()
    try:
        _simulate(memory)
        raise ValueError("Program did not terminate and instruction pointer out of bounds")
    except ProgramStop as stop:
        return stop.state

def _simulate(memory):
    ip = 0  # instruction pointer
    while ip < len(memory):
        instruction = interpreter[memory[ip]]
        params = memory[ip+1 : ip+1 + instruction.n_args]
        ip += 1 + instruction.n_args
        instruction.func(memory, *params)

## Example

In [23]:
program = np.array([1,9,10,3,2,3,11,0,99,30,40,50])

In [26]:
simulate(program)

array([3500,    9,   10,   70,    2,    3,   11,    0,   99,   30,   40,
         50])

## Part 1

In [27]:
program = np.array(list(map(int, lines[0].split(","))))
program[1] = 12
program[2] = 2

resulting_intcodes = simulate(program)

print(resulting_intcodes[0])

3760627


## Part 2

In [28]:
search = 19690720

In [29]:
for noun in range(100):
    for verb in range(100):
        program[1] = noun
        program[2] = verb
        resulting_intcodes = simulate(program)
        if resulting_intcodes[0] == search:
            print(f"Found {search}: Noun={noun}, Verb={verb}")
            break
    if resulting_intcodes[0] == search:
        break

Found 19690720: Noun=71, Verb=95


In [30]:
print(noun*100 + verb)

7195
